# Physics 494/594
## Feature Maps for Non-Linear Functions

In [ ]:
# %load ./include/header.py
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('./include')
import ml4s
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('./include/notebook.mplstyle')
np.set_printoptions(linewidth=120)
ml4s._set_css_style('./include/bootstrap.css')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

## Last Time

### [Notebook Link: 04_Linear_Regression.ipynb](./04_Linear_Regression.ipynb)

- Cost functions and formulating a machine learning task as an optimization problem
- Understand linear regression 

## Today

- Learn how linear regression can learn non-linear functions using feature maps.
- Understanding model complexity and the bias-variance tradeoff

We can generalize everything we learned about linear regression to non-linear models composed of a linear combination of non-linear parts, *i.e.* feature maps.  Let us change our notation slightly (as we would like to be fully general).  As before, our goal is to predict a scalar *target* $y$ as a function of scalar $x$ given a dataset of pairs $\mathcal{D} = \{(x^{(n)},y^{(n)})\}_{n=1}^N$.  Here the $x^{(n)}$ are inputs and the $y^{(n)}$ are targets or observations.  We now let our model be more general:

\begin{equation}
F(x,\vec{w}) = \vec{w}^{\sf T} \vec{\varphi}(x)
\end{equation}

which is **linear** in the *weights*: 

\begin{equation}
\vec{w} = \left( \begin{array}{c}
w_0 \\
w_1 \\
\vdots \\
w_{M-1}
\end{array}
\right)
\end{equation}

but can be **non-linear** in $x$ depending on the basis functions features (think pre-processing layer):

\begin{equation}
\vec{\varphi}(x) = \left( \begin{array}{c}
\varphi_0(x) \\
\varphi_1(x) \\
\vdots \\
\varphi_{M-1}(x)
\end{array}
\right)\, .
\end{equation}

### Examples of Basis Functions
1. **Identity Transformation:** $\varphi_j(x) = x$; this is just the linear regression we have already seen.
2. **Polynomial Decomposition:** $\varphi_j(x) = x^j$; note that the basis functions are global, they behave non-trivially on the entire domain of $x$.
3. **Sigmoid Basis:** $\varphi_j(x) = \sigma((x-\mu_j)/s)$ where $\sigma(z) = 1/(1-\mathrm{e}^{-z})$; note that this can affect a local region of the domain non-trivially.
4. **Fourier  Basis:** $\varphi_j(x) = \mathrm{e}^{i 2\pi x_j}$; action is local in the frequency domain.

Consider a 3rd order polynomial in $x$:

In [ ]:
N = [1,4,1]
labels = [[r'$x$'],['1','$x$','$x^2$','$x^3$'],[r'$F(x,\vec{w})$']]
ml4s.draw_network(N,node_labels=labels, weights=[['' for i in range(N[1])],[f'$w_{i}$' for i in range(N[1])]])

All our previous derivations go through unchanged provided we replace our design matrix $\mathbf{X}$ with a new feature matrix $\mathbf{\Phi}$:

\begin{equation}
\mathbf{\Phi} = \left( \begin{array}{cccc}
        \varphi_0(x^{(1)}) & \varphi_1(x^{(1)}) & \cdots & \varphi_{M-1}(x^{(1)}) \\
        \varphi_0(x^{(2)}) & \varphi_1(x^{(2)}) & \cdots & \varphi_{M-1}(x^{(2)}) \\
\vdots        &      \vdots    & \ddots & \vdots \\
\varphi_0(x^{(N)}) & \varphi_1(x^{(1)}) & \cdots & \varphi_{M-1}(x^{(N)}) \\
\end{array}
\right)
\end{equation}

which, when minimizing the squared error costs across the entire dataset:

\begin{equation}
\boxed{
\mathcal{C} = \frac{1}{2N} \sum_{n=1}^N  \lvert \lvert F^{(n)}(x^{(n)},\vec{w}) - y^{(n)} \rvert \rvert^2
}
\end{equation}

yields the optimal parameters:

\begin{equation}
\vec{w}^\ast = \left(\mathbf{\Phi}^{\sf T} \mathbf{\Phi}\right)^{-1} \mathbf{\Phi}^{\sf T} \vec{y}.
\end{equation}

where $\vec{y}$ is the vector of targets (corresponding to each sample).


## Example

Load data from disk `../include/poly_regression.dat`


<!--
x = np.linspace(0,1,10)
header = f"{'x':>13s}\t{'y':>15s}"
data_out = np.column_stack([x,np.sin(2*np.pi*x)+np.random.normal(loc=0,scale=0.15,size=x.size)])
np.savetxt('../data/poly_regression.dat', data_out,fmt='% 15.8e', header=header, delimiter='\t')
-->

In [ ]:
!head ../data/poly_regression.dat

In [ ]:
x,y = np.loadtxt('../data/poly_regression.dat',unpack=True)
plt.plot(x,y, 'o')
plt.xlabel('x')
plt.ylabel('y')

In [ ]:
poly_order = 3
Φ = np.zeros([len(x),poly_order+1])
for j in range(Φ.shape[1]):
    Φ[:,j] = x**j

In [ ]:
W_opt = np.dot(np.dot(np.linalg.inv(np.dot(Φ.T,Φ)),Φ.T),y)
C_opt = 0.5*np.average((np.dot(Φ,W_opt)-y)**2)

print(f'W_opt = {W_opt}')
print(f'C_opt = {C_opt}')

Again, we can compare this with the `np.polyfit` package

In [ ]:
np.polyfit(x,y,poly_order)

In [ ]:
plt.plot(x,y, 'o', label='data')

x_fit = np.linspace(np.min(x),np.max(x),100)
Φ_fit = np.zeros([len(x_fit),poly_order+1])
for j in range(Φ.shape[1]):
    Φ_fit[:,j] = x_fit**j

plt.plot(x_fit,np.dot(Φ_fit,W_opt),'-', color=colors[0], label='fit')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()

## Exploring Model Complexity

Here we have just guessed the order of the polynomial for regression.  We can systematically explore the fit as a function of the order of the polynomial.

In [ ]:
plt.plot(x,y, 'o', label='data', color=colors[0])
plt.plot(x_fit,np.sin(2*np.pi*x_fit), color=colors[0], ls='--', label=r'$\sin(2\pi x)$')

W_opt = []
C_opt = []

poly_order = [0,4,9]

for n,cpoly_order in enumerate(poly_order):
    Φ = np.zeros([len(x),cpoly_order+1])
    for j in range(Φ.shape[1]):
        Φ[:,j] = x**j
        
    W_opt.append(np.dot(np.dot(np.linalg.inv(np.dot(Φ.T,Φ)),Φ.T),y))
    C_opt.append(0.5*np.average((np.dot(Φ,W_opt[n])-y)**2))
    
    Φ_fit = np.zeros([len(x_fit),cpoly_order+1])
    for j in range(Φ.shape[1]):
        Φ_fit[:,j] = x_fit**j

    plt.plot(x_fit,np.dot(Φ_fit,W_opt[n]),'-', color=colors[2*n+1], label=f'order = {poly_order[n]}; cost = {C_opt[n]:.1e}' )
        
    
plt.legend(loc=(1,0.0))
plt.xlabel('x')
plt.ylabel('y')

In this example we can observe two extremes:

**Underfitting:** Model is too simple – does not fit the data.

**Overfitting:** Model is too complex – fits perfectly but does not generalize.

We can explore this behavior systematically by employing knowledge from **Statistical Learning Theory** (see [Online Course](https://work.caltech.edu/telecourse) and associated textbook by Yaser Abu-Mostafa).

Let us consider our previous example (noisy $\sin(2\pi x)$) but with much more data ($N = 1000$):

\begin{equation}
\mathcal{D} = \{(x^{(n)},y^{(n)})\}_{n=1}^{1000}
\end{equation}


The **first step** is to randomly divide our data set $\mathcal{D}$ into two mutually exclusive groups $\mathcal{D}_{\rm train}$ and $\mathcal{D}_{\rm test}$.  Can do this with built-in `numpy` functions.  No hard-and-fast rule for the relative sizes (we are encountering one of our first *hyperparameters*), 90% is a good place to start.

In [ ]:
x,y = np.loadtxt('../data/poly_regression_long.dat',unpack=True)

N_train = int(0.9*x.size)
indices = np.random.permutation(x.shape[0])
training_idx, test_idx = indices[:N_train], indices[N_train:]
x_train,x_test = x[training_idx],x[test_idx]
y_train,y_test = y[training_idx],y[test_idx]

But it is often simpler to use pre-canned versions from machine-learning libraries

In [ ]:
x,y = np.loadtxt('../data/poly_regression_long.dat',unpack=True)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [ ]:
plt.plot(x_train,y_train, 'o', ms=4, label='Training Data')
plt.plot(x_test,y_test, 's', ms=4, label='Testing Data')

plt.xlabel('x')
plt.ylabel('y')
plt.legend()

We then minimize the cost function using **only** data in the training set.  This gives us two different measurements of error:

#### In-Sample (training) Error
\begin{equation}
E_{\rm in} = \mathcal{C}\left(\vec{y}_{\rm train},F(\vec{x}_{\rm train}, \vec{w}^\ast)\right)
\end{equation}

#### Out-of-sample (testing) Error
\begin{equation}
E_{\rm out} = \mathcal{C}\left(\vec{y}_{\rm test},F(\vec{x}_{\rm test}, \vec{w}^\ast)\right)
\end{equation}

It is **almost always the case** that out-of-sample error is greater than in-sample error.

\begin{equation}
E_{\rm out} \ge E_{\rm in}.
\end{equation}

Splitting the data into mutually exclusive training and test sets provides an unbiased estimate for the predictive performance of the model — this is known as cross-validation in the ML and statistics literature.

In [ ]:
W_opt = []
E_in,E_out = [],[]

poly_order = np.arange(20)

for n,cpoly_order in enumerate(poly_order):
    
    # training
    Φ = np.zeros([len(x_train),cpoly_order+1])
    for j in range(Φ.shape[1]):
        Φ[:,j] = x_train**j
    W_opt.append(np.dot(np.dot(np.linalg.inv(np.dot(Φ.T,Φ)),Φ.T),y_train))
    
    # in-sample (training) error
    E_in.append(0.5*np.average((np.dot(Φ,W_opt[n])-y_train)**2))
    
    # out-of-sample (testing) error
    Φ = np.zeros([len(x_test),cpoly_order+1])
    for j in range(Φ.shape[1]):
        Φ[:,j] = x_test**j
    E_out.append(0.5*np.average((np.dot(Φ,W_opt[n])-y_test)**2))

In [ ]:
plt.plot(poly_order,E_in, label=r'$E_{\rm in}$')
plt.plot(poly_order,E_out, label=r'$E_{\rm out}$')
plt.yscale('log')

plt.xlabel('Polynomial Order (Model Complexity)')
plt.ylabel('Error')
plt.legend()

## Regularized Least Squares

When we enter a regime of **overfitting** one consistent observation is that the fitting parameters tend to grow very large.

In [ ]:
colors = ml4s.get_linear_colors('Spectral_r',len(poly_order))
for i,cW in enumerate(W_opt):
    plt.plot(np.abs(cW), '-o', ms=5, lw=1, color=colors[i])
plt.yscale('symlog')
plt.ylabel(r'$w_j$')
plt.xlabel(r'$j$')

A simple way to address this is just to punish large values of the weights in a modified cost function:

\begin{equation}
C_{\rm ridge}(\vec{w}) = \frac{1}{2}|| \vec{F}(\vec{w})-\vec{y}||^2 + \frac{\lambda}{2} ||\vec{w}||^2
\end{equation}

where $\lambda$ is a *regularization constant*.  This is another hyperparameters that we need to choose by investigation.  Different powers of the regression term can act differently ($||\vec{w}||$ is called *lasso* regression in the statistics literature.  For the choice above, our equation for the optimal parameters above are modified to:

\begin{equation}
\vec{w}^\ast = \left(\lambda \mathbb{1} + \mathbf{\Phi}^{\sf T} \mathbf{\Phi}\right)^{-1} \mathbf{\Phi}^{\sf T} \vec{y}
\end{equation}

where $\mathbb{1}$ is the $M\times M$ identity matrix.  Note the fact that the regularizer now prevents divergences if $\mathbf{\Phi}^{\sf T} \mathbf{\Phi}$ becomes singular.  We can easily modify our code above to use this regularizer.

<div class="span alert alert-success">
<h2> Team Programming challenge: Ridge Regression </h2>

Adapt our analysis above to use use ridge regression.  Play around with different values of $\lambda$ to observe their effects.
</div>